## Instalação do pacote necessário

In [106]:
!pip install rapidfuzz
!pip install pandas
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Importação dos pacotes a serem utilizados

In [2]:
import csv
import os
from rapidfuzz import fuzz
from rapidfuzz.process import extract

## Definição de 

In [3]:
def read_csv(csv_path, delimiter_char):
    with open(csv_path, mode='r',encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=delimiter_char)
        line_count = 0
        wanted_columns = []
        for row in csv_reader:

            if line_count == 0:
                columns_names = delimiter_char.join(row).split(delimiter_char)
                line_count += 1

            data={}
            for column_name in columns_names: 
                data[column_name]=row[column_name]
            wanted_columns.append(data)
            line_count += 1

        return wanted_columns

In [148]:
product_search_name = "tapioca crocante"
search_accuracy = 70
search_result_limit = 5

In [219]:
paths = [os.path.abspath(os.path.join('searched_csvs', p)) for p in os.listdir('searched_csvs')]
array_complete = []
array_matches = []
array_results = []

for path in paths:
    secondary_comparer=read_csv(path,';')

    result_match=None
    result_match2=None
    score1 = extract(product_search_name, [d['ProductName'] for d in secondary_comparer], scorer=fuzz.QRatio,limit=search_result_limit, score_cutoff=search_accuracy)
        
    if score1:
        array_inside=[]
        for score in score1:
            product_name = score[0]
            brand=secondary_comparer[score[2]]['Brand']
            match_score =round(score[1],2)
            csv_name= path.split('/')[-1]

            obj={'Product_Name': product_name,'Brand':brand,'Match_Score':match_score}

            array_inside.append(obj)

            obj["Searched_CSV"]=csv_name
            array_results.append(obj)

        result_match=array_inside
        obj_matches={'Searched_CSV':csv_name,'Match_Result':len(result_match)}
        array_matches.append(obj_matches)

    obj_result={'Searched_CSV':csv_name,'Match_Result':result_match}
    array_complete.append(obj_result)


In [220]:
print(f'Result of Fuzzy String Match of {product_search_name}:')
array_complete

Result of Fuzzy String Match of tapioca crocante:


[{'Searched_CSV': 'MERCADOS_EMPORIOS_E_PADARIAS.csv', 'Match_Result': None},
 {'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv',
  'Match_Result': [{'Product_Name': 'PAPRICA PICANTE',
    'Brand': 'DAFAPS INDUSTRIA E COMERCIO DE PRODS ALIMS. LTDA',
    'Match_Score': 70.97,
    'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'},
   {'Product_Name': 'PAPRICA PICANTE',
    'Brand': 'FECULARIA DE MILHO SAO JOSE LTDA',
    'Match_Score': 70.97,
    'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'},
   {'Product_Name': 'PAPRICA PICANTE',
    'Brand': 'FUCHS GEWURZE DO BRASIL LTDA.',
    'Match_Score': 70.97,
    'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'},
   {'Product_Name': 'PAPRICA PICANTE',
    'Brand': 'MAC TOST IND COM PROD ALIMENTICIO LTDA - ME',
    'Match_Score': 70.97,
    'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'},
   {'Product_Name': 'PAPRICA PICANTE',
    'Brand': 'TEMPEROS & ARTE IND. E COM. LTDA ME',
    'Match_Score': 70.97,
    'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'}]},
 {'Se

In [221]:
print('Number of matches per CSV with Match_Result different than None:')
array_matches

Number of matches per CSV with Match_Result different than None:


[{'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv', 'Match_Result': 5},
 {'Searched_CSV': 'MERCADOS_EMPORIOS_E_PADARIAS.csv', 'Match_Result': 1}]

In [222]:
print('Final Array with Product_Name, Brand, Match_Score and Searched_CSV:')
array_results

Final Array with Product_Name, Brand, Match_Score and Searched_CSV:


[{'Product_Name': 'PAPRICA PICANTE',
  'Brand': 'DAFAPS INDUSTRIA E COMERCIO DE PRODS ALIMS. LTDA',
  'Match_Score': 70.97,
  'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'},
 {'Product_Name': 'PAPRICA PICANTE',
  'Brand': 'FECULARIA DE MILHO SAO JOSE LTDA',
  'Match_Score': 70.97,
  'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'},
 {'Product_Name': 'PAPRICA PICANTE',
  'Brand': 'FUCHS GEWURZE DO BRASIL LTDA.',
  'Match_Score': 70.97,
  'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'},
 {'Product_Name': 'PAPRICA PICANTE',
  'Brand': 'MAC TOST IND COM PROD ALIMENTICIO LTDA - ME',
  'Match_Score': 70.97,
  'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'},
 {'Product_Name': 'PAPRICA PICANTE',
  'Brand': 'TEMPEROS & ARTE IND. E COM. LTDA ME',
  'Match_Score': 70.97,
  'Searched_CSV': 'DADOS_ABERTOS_ALIMENTO.csv'},
 {'Product_Name': 'TAPIOCA CROCANTE 180g',
  'Brand': 'Tia Sônia',
  'Match_Score': 86.49,
  'Searched_CSV': 'MERCADOS_EMPORIOS_E_PADARIAS.csv'}]

In [189]:
import pandas as pd

In [190]:
df = pd.DataFrame.from_records(array_matches)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Searched_CSV  2 non-null      object
 1   Match         2 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 160.0+ bytes


In [191]:
df.head()

,Searched_CSV,Match
0,DADOS_ABERTOS_ALIMENTO.csv,5
1,MERCADOS_EMPORIOS_E_PADARIAS.csv,1


In [192]:
df.dropna(inplace=True)

In [193]:
df.head()

,Searched_CSV,Match
0,DADOS_ABERTOS_ALIMENTO.csv,5
1,MERCADOS_EMPORIOS_E_PADARIAS.csv,1
